In [ ]:
!pip install pytelegrambotapi

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import telebot
import torchaudio


In [3]:
cd "/content/drive/MyDrive/audio_classification"

/content/drive/MyDrive/audio_classification


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

Using cuda


## Network

#Bot

In [5]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=11, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.softmax(x, dim=1)
        return x


In [6]:
model = M5(n_output=11).to(device)
model.load_state_dict(torch.load('weights_5.pth'))

labels = ['mohammadali', 'morteza', 'zeynab', 'alireza', 'maryam', 'nahid', 'parisa', 'zahra', 'sajjad', 'hosein', 'amir']

In [ ]:
bot = telebot.TeleBot('5842423589:AAHTZWfT5wcGtqPumQJTPWKg05Kji1VWggo')
@bot.message_handler(commands=['start'])
def my_start(message):
  bot.send_message(message.chat.id,f'hello, welcome {message.from_user.first_name} to voice recognition bot.')
  bot.send_message(message.chat.id,'send me your voice 🎤 ')

@bot.message_handler(content_types=['voice'])
def my_voice(message):
  file_info = bot.get_file(message.voice.file_id)
  downloaded_file = bot.download_file(file_info.file_path)
  path = file_info.file_path

  with open(path, 'wb') as audio_file:
    audio_file.write(downloaded_file)
  
  # Inference
  signal, sample_rate = torchaudio.load(path)

  # preprocess
  signal = torch.mean(signal, dim=0, keepdim=True)
  new_sample_rate = 8000
  transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
  signal = transform(signal)
  # signal = signal[:, 32000:40000]
  signal = signal.unsqueeze(0).to(device)

  # process
  preds = model(signal)

  # postprocess
  preds = preds.cpu().detach().numpy()
  print(preds)
  output = np.argmax(preds)
  print("Mr/Miss", labels[output],output)
  bot.reply_to(message,"Mr/Mrs "+ labels[output])


bot.polling()
     